In [1]:
import sys
import os
import re
import pandas as pd
import numpy as np
from PyQt5.QtWidgets import QApplication, QFileDialog, QMessageBox

def select_directory(title):
    app = QApplication(sys.argv)
    folder = QFileDialog.getExistingDirectory(None, title)
    if not folder:
        QMessageBox.critical(None, 'Error', f"No folder selected for {title.lower()}! Exiting.")
        sys.exit()
    return folder

def get_file_path(folder, file_suffix):
    return [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(file_suffix)]

def read_max_pld_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        data_rows = [line.strip().split('|')[1:3] for line in lines[2:] if line.strip()]
    df_intf = pd.DataFrame(data_rows, columns=['Interface Labels', 'Units'])
    return df_intf.T

def insert_phase_columns(df):
    new_df = pd.DataFrame()
    for col in df.columns:
        new_df[col] = df[col]
        new_col_label = 'Phase_' + df[col].iloc[0].strip()
        new_df[new_col_label] = [new_col_label, 'deg']
    return new_df

def read_pld_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        headers = [h.strip() for h in lines[0].strip().split('|')[1:-1]]
        processed_data = []
        for line in lines[2:]:  # Skip header rows
            line = line.strip()
            if not line.startswith('|'):
                line = '|' + line  # Prepend '|' if missing
            if not line.endswith('|'):
                line = line + '|'  # Append '|' if missing
            # Now process the line assuming it is correctly formatted
            data_cells = [float(re.sub('[^0-9.E-]', '', cell.strip())) for cell in line.split('|')[1:-1]]
            processed_data.append(data_cells)
    return pd.DataFrame(processed_data, columns=headers)

def main():
    folder_selected_raw_data = select_directory('Please select a directory for raw data')
    folder_selected_headers_data = select_directory('Please select a directory for data headers')

    file_path_full_data = get_file_path(folder_selected_raw_data, 'full.pld')
    file_path_headers_data = get_file_path(folder_selected_headers_data, 'max.pld')

    if not file_path_full_data or not file_path_headers_data:
        QMessageBox.critical(None, 'Error', "No required files found! Exiting.")
        sys.exit()

    dfs = [read_pld_file(file_path) for file_path in file_path_full_data]
    df = pd.concat(dfs, ignore_index=True)

    df_intf_before = read_max_pld_file(file_path_headers_data[0])
    df_intf = insert_phase_columns(df_intf_before)

    df_intf_labels = pd.DataFrame(df_intf.iloc[0]).T
    new_columns = ['FREQ'] + df_intf_labels.iloc[0].tolist()
    df.columns = new_columns[:len(df.columns)]  # Adjust columns if mismatched
    df.columns = [col.strip() for col in new_columns[:len(df.columns)]]

    print(df)

    return df

if __name__ == "__main__":
    data=main()

C:\Users\emre_\AppData\Local\Temp\ipykernel_10344\2675821076.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


    FREQ  I1 - STBD REAR MOUNT (CS-8012) T1  \
0    1.0                              697.0   
1    2.0                              697.5   
2    3.0                              698.0   
3    4.0                              698.5   
4    5.0                              699.0   
5    6.0                              699.5   
6    7.0                              700.0   
7    8.0                              700.5   
8    9.0                              701.0   
9   10.0                              701.5   
10  11.0                              702.0   
11  12.0                              702.5   
12  13.0                              703.0   
13  14.0                              703.5   
14  15.0                              704.0   
15  16.0                              704.5   
16  17.0                              705.0   
17  18.0                              705.5   
18  19.0                              706.0   
19  20.0                              706.5   
20   1.0     

In [2]:
data

,FREQ,I1 - STBD REAR MOUNT (CS-8012) T1,Phase_I1 - STBD REAR MOUNT (CS-8012) T1,I2 - PORT REAR MOUNT (CS-8013) T1,Phase_I2 - PORT REAR MOUNT (CS-8013) T1,I3-TT/FBS/IPS - TT Side (CS-8001) T1,Phase_I3-TT/FBS/IPS - TT Side (CS-8001) T1,I3-TT/FBS/IPS - TT Side (CS-8001) T2,Phase_I3-TT/FBS/IPS - TT Side (CS-8001) T2,I3-TT/FBS/IPS - TT Side (CS-8001) T3
0,1.0,697.0,2.016850,47.27343,1.220497,-57.0614,0.369561,-50.30029,0.845223,93.63654
1,2.0,697.5,2.013314,47.34493,1.210233,-57.8759,0.378650,-49.84751,0.846367,93.55009
2,3.0,698.0,2.010290,47.41929,1.200017,-58.7065,0.388132,-49.46666,0.847604,93.47648
3,4.0,698.5,2.007733,47.49436,1.189841,-59.5477,0.397974,-49.15783,0.848946,93.40883
4,5.0,699.0,2.005616,47.56886,1.179709,-60.3962,0.408144,-48.92019,0.850398,93.34330
5,6.0,699.5,2.003919,47.64195,1.169628,-61.2497,0.418608,-48.75248,0.851962,93.27757
6,7.0,700.0,2.002629,47.71303,1.159609,-62.1063,0.429329,-48.65292,0.853638,93.21017
7,8.0,700.5,2.001736,47.78162,1.149666,-62.9647,0.440271,-48.61907,0.855426,93.14009
8,9.0,701.0,2.001232,47.84733,1.139817,-63.8235,0.451400,-48.64812,0.857327,93.06664
9,10.0,701.5,2.001108,47.90982,1.130081,-64.6817,0.462678,-48.73687,0.859339,92.98929


In [3]:
data.to_csv("anan.csv", index=False)

In [4]:
# Load the CSV file into a DataFrame
df = pd.read_csv('anan.csv')

# Columns for the first plot (without 'Phase_' in them but otherwise identical)
non_phase_columns = [col for col in df.columns if not col.startswith('Phase_')]
# Columns for the second plot (with 'Phase_' in them)
phase_columns = [col for col in df.columns if col.startswith('Phase_')]

# Determine the corresponding non-phase columns for the phase columns
matched_columns = []
for phase_col in phase_columns:
    potential_match = phase_col.replace('Phase_', '')
    if potential_match in non_phase_columns:
        matched_columns.append(potential_match)

# Data for the first plot
df_first_plot = df[matched_columns]
# Data for the second plot
df_second_plot = df[phase_columns]

In [5]:
import sys
import pandas as pd
from PyQt5 import QtWidgets, QtCore, QtWebEngineWidgets
from PyQt5.QtWidgets import QApplication, QVBoxLayout, QWidget, QTabWidget, QSplitter
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class PlotlyGraphs(QWidget):
    def __init__(self, parent=None):
        super(PlotlyGraphs, self).__init__(parent)
        self.initUI()

    def initUI(self):
        # Create the tab widget and tabs
        tab_widget = QTabWidget(self)
        tab1 = QWidget()
        tab2 = QWidget()

        # Set the layout for the main widget
        main_layout = QVBoxLayout(self)
        self.setLayout(main_layout)

        # Create a splitter to manage the plots' layout in tab1
        splitter = QSplitter(QtCore.Qt.Vertical)
        self.phase_plot = QtWebEngineWidgets.QWebEngineView()
        self.regular_plot = QtWebEngineWidgets.QWebEngineView()

        # Add the plot views to the splitter
        splitter.addWidget(self.regular_plot)
        splitter.addWidget(self.phase_plot)

        # Set the initial sizes of the plots
        splitter.setSizes([self.height() // 2, self.height() // 2])

        # Add the splitter to tab1's layout
        layout_tab1 = QVBoxLayout(tab1)
        tab1.setLayout(layout_tab1)
        layout_tab1.addWidget(splitter)

        # Load and plot data in tab1
        self.load_data_and_plot()

        # Add tabs to the tab widget
        tab_widget.addTab(tab1, "Plots")
        tab_widget.addTab(tab2, "Future Use")

        # Add the tab widget to the main layout
        main_layout.addWidget(tab_widget)

        # Set window title and show maximized
        self.setWindowTitle("Plotly Graphs Viewer")

    def load_data_and_plot(self):
        df = pd.read_csv('anan.csv')

        # Filter out columns with 'Phase_' and without
        phase_columns = [col for col in df.columns if 'Phase_' in col]
        regular_columns = [col for col in df.columns if 'Phase_' not in col]

        # Define layout for the plots
        layout = go.Layout(
            margin=dict(l=20, r=20, t=35, b=35),  # Adjust margins to fit the layout
            legend=dict(font=dict(size=8)),  # Reduce the font size of the legend
        )

        # Plotting the regular data
        fig_reg = make_subplots(rows=1, cols=1)
        for col in regular_columns:
            fig_reg.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col))
        fig_reg.update_layout(layout)

        # Plotting the phase data
        fig_phase = make_subplots(rows=1, cols=1)
        for col in phase_columns:
            fig_phase.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col))
        fig_phase.update_layout(layout)

        # Converting Plotly figures to HTML with a dynamic layout and loading them into QWebEngineViews
        html_reg = fig_reg.to_html(full_html=False, include_plotlyjs='cdn', config={'responsive': True})
        html_phase = fig_phase.to_html(full_html=False, include_plotlyjs='cdn', config={'responsive': True})
        self.regular_plot.setHtml(html_reg)
        self.phase_plot.setHtml(html_phase)

    def showMaximized(self):
        # Override showMaximized to ensure that the window is maximized after the UI has been set up
        super().showMaximized()

if __name__ == "__main__":
    QtWidgets.QApplication.setAttribute(QtCore.Qt.AA_EnableHighDpiScaling, True)
    QtCore.QCoreApplication.setAttribute(QtCore.Qt.AA_ShareOpenGLContexts)
    app = QApplication(sys.argv)
    main = PlotlyGraphs()
    main.showMaximized()  # Maximize window on start
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\emre_\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3556: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

